In [1]:
import configparser
config = configparser.RawConfigParser()
config.read("../api_key.ini")

['../api_key.ini']

In [2]:
print(config.sections())

['API_KEY']


In [3]:
import cohere
API_KEY = config[config.sections()[0]]['api_key']
co = cohere.Client(API_KEY)

In [6]:
import langchain
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
chat = co.chat(
    message="hello world!",
    model="command"
)

print(chat)

text="Hi there! I'm Coral, an AI-assistant chatbot trained to assist human users by providing helpful responses. How can I help you today?\nI'm excited to talk with you and help however I can!" generation_id='0c84eefc-1e2b-4b72-bdc2-94d05276cef1' citations=None documents=None is_search_required=None search_queries=None search_results=None finish_reason='COMPLETE' tool_calls=None chat_history=[ChatMessage(role='USER', message='hello world!'), ChatMessage(role='CHATBOT', message="Hi there! I'm Coral, an AI-assistant chatbot trained to assist human users by providing helpful responses. How can I help you today?\nI'm excited to talk with you and help however I can!")] meta=ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(input_tokens=54, output_tokens=43, search_units=None, classifications=None), tokens=ApiMetaTokens(input_tokens=65, output_tokens=43), warnings=None) response_id='816e76b8-559d-4286-9172-25b6b8f75b